In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#2. specify parameters
pipeline_params={
}
step_params={
}
substep_params={
    "FILTER_EMPTY_GT"    : False,
    "MIN_OBJECT_SIZE"    : 5
}

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
    outputs = 
    [
        { ENTITY_NAME: "train_coco_data"},
        { ENTITY_NAME: "eval_coco_data"},
        { ENTITY_NAME: "cache_config"}
    ],
    tmp_inputs =
    [
        {ENTITY_NAME: "cache_data"},
        {ENTITY_NAME: "cache_config"}
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
#4 write outputs
outputs = substep.outputs()
tmp_inputs = substep.tmp_inputs()

In [ ]:
#5 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

In [ ]:
print(f"{outputs.train_coco_data=}")
print(f"{outputs.eval_coco_data=}")
print(f"{outputs.cache_config=}")

In [ ]:
print(f"{tmp_inputs.cache_data=}")
print(f"{tmp_inputs.cache_config=}")

### Check annotations

In [ ]:
from utils.coco import join_coco_files, load as load_coco
from utils.coco import preview_coco_file
from utils.coco import show_item
from utils.coco import get_dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import os.path as osp
import os

In [ ]:
coco_annotation = load_coco(osp.join(tmp_inputs.cache_data, "annotations", "instances_val2017.json"))

In [ ]:
coco_annotation.keys()

In [ ]:
# Selection of object types for subsequent neural network training
select_object_names = ["person", "bicycle", "car", "motorcycle", "bus", "truck"]
CLASSES = select_object_names
select_categories= [cat_info.copy() for cat_info in coco_annotation["categories"] if cat_info["name"] in select_object_names]
for new_id, cat_info in enumerate(select_categories, 1):
    cat_info["old_id"] = cat_info["id"]
    cat_info["id"] = new_id 
select_categories

In [ ]:
# Select annotation object by select_categories
reid_categories_ids = {cat_info["old_id"]: cat_info["id"] for cat_info in select_categories} # reidentification categories

new_coco_annotations = []
for annot in coco_annotation["annotations"]:
    new_annot = annot.copy()
    category_id = new_annot["category_id"]
    if category_id in reid_categories_ids.keys():
        new_annot["category_id"] = reid_categories_ids[category_id]
        new_coco_annotations.append(new_annot)

In [ ]:
# apply new annotation
coco_annotation["categories"] = select_categories.copy()
coco_annotation["annotations"] = new_coco_annotations.copy()

In [ ]:
# checking segmentation in annotations (list in segmentation)
bad_image_ids = [annot["image_id"] for annot in coco_annotation["annotations"] if not isinstance(annot["segmentation"], list)]

true_coco_images = [img_info for img_info in coco_annotation["images"] if not img_info["id"] in bad_image_ids]
true_coco_annotations = [annot for annot in coco_annotation["annotations"] if not annot["image_id"] in bad_image_ids]
coco_annotation["images"] = true_coco_images.copy()
coco_annotation["annotations"] = true_coco_annotations.copy()

In [ ]:
# checking the existence of annotations for an image

coco_image_ids =  [img_info["id"] for img_info in coco_annotation["images"]]
coco_annotations_image_ids = [annot["image_id"] for annot in coco_annotation["annotations"]]

bad_image_ids = list(set(np.unique(coco_image_ids)) - set(np.unique(coco_annotations_image_ids)))

true_coco_images = [img_info for img_info in coco_annotation["images"] if not img_info["id"] in bad_image_ids]
true_coco_annotations = [annot for annot in coco_annotation["annotations"] if not annot["image_id"] in bad_image_ids]

coco_annotation["images"] = true_coco_images.copy()
coco_annotation["annotations"] = true_coco_annotations.copy()

In [ ]:
len(coco_annotation["images"])

### Split Dataset to Train and Valid

In [ ]:
# split to train and valid parts
train_coco_images, val_coco_images = train_test_split(coco_annotation["images"], test_size=0.33, random_state=42)
train_images_ids = [img_info["id"] for img_info in train_coco_images]
val_images_ids = [img_info["id"] for img_info in val_coco_images]
train_images_names = [img_info["file_name"] for img_info in train_coco_images]
val_images_names = [img_info["file_name"] for img_info in val_coco_images]

In [ ]:
train_coco_annotations = [annot.copy() for annot in coco_annotation["annotations"] if annot["image_id"] in train_images_ids]
val_coco_annotations = [annot.copy() for annot in coco_annotation["annotations"] if annot["image_id"] in val_images_ids]

In [ ]:
train_coco = coco_annotation.copy()
train_coco["images"] = train_coco_images
train_coco["annotations"] = train_coco_annotations

val_coco = coco_annotation.copy()
val_coco["images"] = val_coco_images
val_coco["annotations"] = val_coco_annotations

In [ ]:
# preview data for train and valid dataset
preview_coco_file(train_coco, img_folder=osp.join(tmp_inputs.cache_data, "val2017"), count=2)
preview_coco_file(val_coco, img_folder=osp.join(tmp_inputs.cache_data, "val2017"), count=2)

In [ ]:
import matplotlib.pyplot as plt

areas  = []
counts = []
categories = []
categories_annotation = []

anns = coco_annotation.get('annotations', [])
for image in coco_annotation.get('images', []):
    image_anns = [ann for ann in anns if ann['image_id'] == image['id']]
    counts.append(len(image_anns))
    
    for ann in image_anns:
        areas.append(ann.get('area'))
        categories.append(ann.get('category_id'))
    
    categories_annotation += coco_annotation['categories']
           
counts = np.array(counts)
areas  = np.array(areas)

In [ ]:
import plotly.express as px

fig = px.histogram(areas, title='Площать объектов на фотографиях датасета')
fig.layout.yaxis.title = 'Кол-во объектов'
fig.layout.xaxis.title = 'Площадь'
fig.show()

In [ ]:
fig = px.histogram(counts, title='Кол-во объектов на фотографиях датасета')
fig.layout.yaxis.title = 'Кол-во объектов'
fig.layout.xaxis.title = 'Кол-во фотографий'
fig.show()

## Save train and validation datasets to outputs

In [ ]:
### Save train and validation datasets to parquet

import json
from pathlib import Path
from tqdm import tqdm
import pandas as pd

def build_pandas_df(_coco_data, img_folder:str = ""):
    pack = []
    for img_info in tqdm(_coco_data["images"]):
        row = {
            "file_names"       : None,
            "files_binary"     : None,
        }
        file_name = osp.join(img_folder, img_info["file_name"])
        with open(file_name, 'rb') as file_fd:
            row['files_binary'] = file_fd.read()
        row["file_names"] = img_info["file_name"]
        pack.append(row.copy())
    return pack

In [ ]:
data_train_df =  build_pandas_df(train_coco, img_folder=osp.join(tmp_inputs.cache_data, "val2017"))
data_train_df = spark.createDataFrame(data_train_df)
data_train_df.printSchema()

data_val_df =  build_pandas_df(val_coco, img_folder=osp.join(tmp_inputs.cache_data, "val2017"))
data_val_df = spark.createDataFrame(data_val_df)
data_val_df.printSchema()

In [ ]:
data_train_df.write.mode("overwrite").option("compression", "none").parquet(outputs.train_coco_data)
data_val_df.write.mode("overwrite").option("compression", "none").parquet(outputs.eval_coco_data)

In [ ]:
# save train and valid annotations to json
os.makedirs(outputs.cache_config, exist_ok=True)
train_annotation_path = osp.join(tmp_inputs.cache_config, "train_coco_annotations.json")
val_annotation_path = osp.join(tmp_inputs.cache_config, "val_coco_annotations.json")

with open(train_annotation_path, 'w') as f:
    json.dump(train_coco, f, indent=4)

with open(val_annotation_path, 'w') as f:
    json.dump(val_coco, f, indent=4)
    
CONFIG = dict(**substep_params)
CONFIG["train_coco_annotation"] = "train_coco_annotations.json"
CONFIG["val_coco_annotation"] = "val_coco_annotations.json"
CONFIG["train_images"] = "train_coco_data"
CONFIG["val_images"] = "eval_coco_data"
CONFIG["CLASSES"] = CLASSES

config_path = osp.join(tmp_inputs.cache_config, "config.json")
with open(config_path, 'w') as f:
    json.dump(CONFIG, f, indent=4)

In [ ]:
# saves tmp_outputs.cache_config (json files of configs) to outputs as parquets
from sinara.store import SinaraStore

SinaraStore.copy_tmp_files_to_store(tmp_dir=tmp_inputs.cache_config, store_dir=outputs.cache_config, file_globs=["*"])

In [ ]:
#7 stop spark
SinaraSpark.stop_session()